In [13]:
import os, math
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.datasets import cifar10, mnist
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization 
from tensorflow.keras.layers import Conv2D, Flatten, Activation
from tensorflow.keras.layers import Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mse, SparseCategoricalCrossentropy

In [7]:
(x_train, y_train),(x_test, _) = mnist.load_data()


In [11]:
x_train = x_train.reshape(-1, 28, 28, 1)


In [3]:
image_size = x_train[0].shape[1]
image_size

28

In [41]:
class ConvTransBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()
        self.bn = BatchNormalization()  # check training flag
        self.act = Activation(activation='relu')
        self.conv2D_trans = Conv2DTranspose(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='same')
        
    def call(self, inputs, training=False):
        x = self.bn(inputs)
        x = self.act(x)
        return self.conv2D_trans(x)


In [ ]:
noise = np.random.uniform(0, 1, size=[10000, 100])


In [ ]:
model = Sequential()
model.add(Input(noise.shape[1]))
model.add(Dense(14*14*32, activation="relu"))
model.add(Reshape([14, 14, 32]))
model.add(ConvTransBlock(32, 5, 2))
model.add(ConvTransBlock(16, 5, 1))
model.add(Conv2DTranspose(1, 5, padding="same", activation="sigmoid"))


In [ ]:
model.compile(loss="MSE", metrics=["MAE"])
model.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000])

In [39]:
class Generator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size, resize_img):
        super().__init__()
        self.dense1 = Dense(resize_img * resize_img * filters[0])
        self.reshape = Reshape([resize_img, resize_img, filters[0]])
        self.conv2dtrans = []
        for i, _filter in enumerate(filters):
            if i <= 1:
                strides = 2
            else:
                strides = 1
            self.conv2dtrans.append(ConvTransBlock(_filter, kernel_size, strides))
        
        self.act = Activation("sigmoid")

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.reshape(x)
        for conv in self.conv2dtrans:
            x = conv(x)
        return self.act(x)




In [ ]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

In [ ]:
gen.compile(loss="mse", metrics=["mae"])

In [ ]:
gen.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000], batch_size=64)

In [9]:
def leaky_conv(filters, kernel_size, strides):
    return Sequential([LeakyReLU(alpha=0.2),
                       Conv2D(filters=filters,
                       kernel_size=kernel_size,
                       strides=strides,
                       padding='same')])


In [19]:
model = Sequential()
model.add(Input(x_train.shape[1:]))
model.add(leaky_conv(32, 5, 2))
model.add(leaky_conv(64, 5, 2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("softmax"))

In [20]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_10 (Sequential)   (None, 14, 14, 32)        832       
_________________________________________________________________
sequential_11 (Sequential)   (None, 7, 7, 64)          51264     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                31370     
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 83,466
Trainable params: 83,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train)

In [24]:
class Discriminator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.leaky_convs = []
        for i, _filter in enumerate(filters):
            if i < len(filters) - 1:
                strides = 2
            else:
                strides = 1
            self.leaky_convs.append(leaky_conv(_filter, kernel_size, strides))
        self.flat = Flatten()
        self.dense = Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = inputs
        for conv in self.leaky_convs:
            x = conv(x)
        x = self.flat(x)
        return self.dense(x)

In [26]:
disc = Discriminator(disc_layers_filters, disc_kernel_size)

In [34]:
disc.compile(loss="categorical_crossentropy", metrics=["mae"])

In [ ]:
disc.fit(x_train, np.where(y_train < 5, 1, 0))

In [36]:
disc.summary()

Model: "discriminator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_12 (Sequential)   (32, 14, 14, 32)          832       
_________________________________________________________________
sequential_13 (Sequential)   (32, 7, 7, 64)            51264     
_________________________________________________________________
sequential_14 (Sequential)   (32, 4, 4, 128)           204928    
_________________________________________________________________
sequential_15 (Sequential)   (32, 4, 4, 256)           819456    
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  4097      
Total params: 1,080,577
Trainable params: 1,080,577
Non-trainable params: 0
_________________________________________

In [37]:
latent_size = 100
batch_size = 64
train_steps = 40000
disc_lr = 2e-4
disc_decay = 6e-8
gen_lr = disc_lr//2
gen_decay = disc_decay//2


In [ ]:
disc_kernel_size = 5
disc_layers_filters = [32, 64, 128, 256]

In [ ]:
gen_resize_img = image_size // 4
gen_kernel_size = 5
gen_layers_filter = [128, 64, 32, 1]

In [44]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

In [45]:
disc = Discriminator(disc_layers_filters, disc_kernel_size)
disc.compile(loss='binary_crossentropy',
             optimizer=RMSprop(learning_rate=disc_lr, decay=disc_decay),
             metrics=['accuracy'])

In [46]:
def build_gan()
gan = Sequential()
gan.add(Input([100,]))
gan.add(gen)
gan.add(disc)

In [49]:
gan.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_2 (Generator)      (None, 28, 28, 1)         1301505   
_________________________________________________________________
discriminator_4 (Discriminat (None, 1)                 1080577   
Total params: 2,382,082
Trainable params: 2,381,378
Non-trainable params: 704
_________________________________________________________________
